In [1]:
from tensorflow import keras
from tensorflow.keras import layers

vocabulary_size = 10000
num_tags = 100
num_departments = 4

#defining model inputs
"""This section just defines what the model will expect as input"""
title = keras.Input(shape=(vocabulary_size,), name="title")
text_body = keras.Input(shape=(vocabulary_size,), name="text_body")
tags = keras.Input(shape=(num_tags,), name="tags")

class CustomerTicketModel(keras.Model):
    def __init__(self, num_departments):
        super().__init__()
        #define sublayers in the constructor
        self.concat_layer = layers.Concatenate()
        self.mixing_layer = layers.Dense(64, activation="relu")
        self.priority_scorer = layers.Dense(1, activation="sigmoid")
        self.department_classifier = layers.Dense(num_departments, activation="softmax")

    def call(self, inputs):
        title = inputs["title"]
        text_body = inputs["text_body"]
        tags = inputs["tags"]

        features = self.concat_layer([title, text_body, tags])
        features = self.mixing_layer(features)
        priority = self.priority_scorer(features)
        department = self.department_classifier(features)
        return priority, department

import numpy as np

num_samples = 1280

#dummy input data
"""Here are the real inputs"""
title_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
text_body_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size)) 
tags_data = np.random.randint(0, 2, size=(num_samples, num_tags)) 

#dummy target data
priority_data = np.random.random(size=(num_samples, 1))
department_data = np.random.randint(0, 2, size=(num_samples, num_departments))

#instantiating the model
model = CustomerTicketModel(num_departments=4)
priority, departments = model({"title": title_data, "text_body": text_body_data, "tags": tags_data})

#Training the model
model.compile(optimizer = "rmsprop",
              loss=["mean_squared_error", "categorical_crossentropy"],
              metrics=[["mean_absolute_error"], ["accuracy"]]
)
model.fit({"title": title_data, "text_body": text_body_data, "tags": tags_data},
         [priority_data, department_data],
         epochs=1)
model.evaluate({"title": title_data, "text_body": text_body_data, "tags": tags_data},
              [priority_data, department_data])
priority_preds, department_preds = model.predict({"title": title_data, "text_body": text_body_data, "tags": tags_data})

        

40/40 [==============================] - 0s 5ms/step
